In [1]:
import pandas as pd
import datetime
import re
import requests
from requests import get
from bs4 import BeautifulSoup
import warnings
from time import sleep
from random import randint
from time import time
timestart_time = time()
warnings.warn('Warning Simulation')
from IPython.core.display import clear_output

C:\Users\k.kiki\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Warning Simulation
  if sys.path[0] == '':


In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
raw_html = requests.get('https://eody.gov.gr/category/deltia-typoy/')
soup = BeautifulSoup(raw_html.content, "html.parser") 
articles_container = soup.find('div',{'class':'articles-container'})
titles = articles_container.find_all('a', href=True)

# if r'νακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας' in titles[0].text:
#     print(titles[0].text,titles[0].get('href'))
# else:
#     print(titles[0].text,titles[0].get('href')+', but you do not care about this announcement!!!')

table = []

for t in titles:
    entries = {}
    
    if 'νακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας' in t.text\
        or 'νημέρωση διαπιστευμένων συντακτών υγείας από' in t.text:

        entries['title'] = t.text
        entries['url'] = t.get('href')

        table.append(entries)
    
table = pd.DataFrame(table)
table['date'] = table.title.str.extract(r'[(](\d+\/\d+\/\d+)[)]')

table.head()

,title,url,date
0,Ανακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας (18/5/2020),https://eody.gov.gr/anakoinosi-gia-tin-exelixi-tis-nosoy-covid-19-sti-chora-mas-18-5-2020/,18/5/2020
1,"Ενημέρωση διαπιστευμένων συντακτών υγείας από τον Υφυπουργό Πολιτικής Προστασίας και Διαχείρισης Κρίσεων Νίκο Χαρδαλιά, τον Παθολόγο-Λοιμωξιολόγο, Υποπτέραρχο Δημήτρη Χατζηγεωργίου και τον Υφυπουργό Ανάπτυξης και Επενδύσεων Νίκο Παπαθανάση (17/5/2020)",https://eody.gov.gr/0517_briefing_covid19/,17/5/2020
2,Ανακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας (16/5/2020),https://eody.gov.gr/anakoinosi-gia-tin-exelixi-tis-nosoy-covid-19-sti-chora-mas-16-5-2020/,16/5/2020
3,"Ενημέρωση διαπιστευμένων συντακτών υγείας από τον Υφυπουργό Πολιτικής Προστασίας και Διαχείρισης Κρίσεων Νίκο Χαρδαλιά και τον εκπρόσωπο του Υπουργείου Υγείας για το νέο κορονοϊό, Καθηγητή Σωτήρη Τσιόδρα",https://eody.gov.gr/enimerosi-diapisteymenon-syntakton-ygeias-apo-ton-yfypoyrgo-politikis-prostasias-kai-diacheirisis-kriseon-niko-chardalia-kai-ton-ekprosopo-toy-ypoyrgeioy-ygeias-gia-to-neo-koronoio-kathigiti-sotir-7/,NaN
4,"Eνημέρωση διαπιστευμένων συντακτών υγείας από τον Υφυπουργό Πολιτικής Προστασίας και Διαχείρισης Κρίσεων Νίκο Χαρδαλιά και τον εκπρόσωπο του Υπουργείου Υγείας για το νέο κορονοϊό, Καθηγητή Σωτήρη Τσιόδρα (14/5/2020)",https://eody.gov.gr/0514_briefing_covid19/,14/5/2020


In [4]:
def get_press_release(row):
    page = requests.get(row['url'])
    soup_doc = BeautifulSoup(page.content, "html.parser") 
    
    par = ''
    for p in soup_doc.find_all('p'):
        par = par+p.text.strip()
        
    return pd.Series({
    'content': par 
    })   

In [5]:
df = table.apply(get_press_release, axis=1).join(table)
# df.head()

In [6]:
df['new_cases'] = df.content.str.extract(r'.*[aα]νακοινώνο\w+\s+(\d+)\s+νέα\s+κρούσματα',re.IGNORECASE).astype(float)
df['total_cases'] = df.content.str.extract(r'συνολικός\s+αριθμός.*κρουσμάτων\s+είναι\s+(\d+)',re.IGNORECASE).astype(float)
df['new_deaths'] = df.content.str.extract(r'(?:μεθ.*έχουμε\s+)(.*)(?:\s+ακόμα\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*και)',re.IGNORECASE)

In [7]:
# d = { "έναν": "1", "δύο": "2", 'τρεις':'3', 'τέσσερις':'4'}
df['new_deaths'] = df.new_deaths.replace({ "έναν": "1", "δύο": "2", 'τρεις':'3', 'τέσσερις':'4'}).astype(float)

In [8]:
df['total_deaths'] = df.content.str.extract(\
                    r'(?:\s+ακόμα\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*και.*?\s+)(\d+).*?\s+θ[άα]ν[αά]τους.*?\s+στη\s+χώρα',\
                                           re.IGNORECASE).astype(float)

In [9]:
df['intubated'] = df.content.str.extract(r'(\d+)\s+συμπολίτες\s+μας\s+νοσηλεύονται\s+διασωληνωμένοι').astype(float)

In [10]:
# df

In [11]:
# df.intubated.value_counts(dropna=False)

In [12]:
# df[df.total_deaths.isnull()]

In [13]:
greece = pd.read_csv('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/greece.csv')
# greece.head()

In [14]:
if greece.cases.sum() < df.total_cases[0] and greece.cases.sum()+df.new_cases[0] == df.total_cases[0]:
    greece[greece['county_en'] == 'Unknown']['cases'] = greece[greece['county_en'] == 'Unknown']['cases']+(df.total_cases[0]-greece.cases.sum())
    greece.to_csv('greece_auto_test.csv')
    print('greece.cases was updated and greece_auto_test.csv has been saved')
    
elif greece.cases.sum() < df.total_cases[0] and greece.cases.sum()+df.new_cases[0] != df.total_cases[0]:
    print('Be careful: Something goes wrong with cases announced!!!')
    
else:
    print('Cool: greece.cases is updated!')
    

    
if greece.dead.sum() != df.total_deaths[0] and greece.dead.sum()+df.new_deaths[0] == df.total_deaths[0]:
    greece[greece['county_en'] == 'Unknown']['dead'] = greece[greece['county_en'] == 'Unknown']['cases']+df.new_deaths[0]
    greece.to_csv('greece_auto_test.csv')
    print('greece.dead was updated and greece_auto_test.csv has been saved')
    
elif greece.dead.sum() != df.total_deaths[0] and greece.dead.sum()+df.new_deaths[0] != df.total_deaths[0]:
    print('Be careful: Something goes wrong with deaths announced!!!')
    
else:
    print('Cool: greece.dead is updated!')
    

if greece.critical.sum() != df.intubated[0]:
    greece[greece['county_en'] == 'Unknown']['critical'] = greece[greece['county_en'] == 'Unknown']['critical']+(df.intubated[0]-greece.critical.sum())
    greece.to_csv('greece_auto_test.csv')
    print('greece.critical was updated and greece_auto_test.csv has been saved')
    
elif greece.critical.sum() == df.intubated[0]:
    print('Cool: greece.critical is updated!')


Cool: greece.cases is updated!
Cool: greece.dead is updated!
greece.critical was updated and greece_auto_test.csv has been saved


C:\Users\k.kiki\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
